IMPORTING LIBRARIES

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime


LOADING DATASET

In [3]:
x_train = pd.read_csv("C:/Users/User/Downloads/Credit Card Fraud  Dataset/fraudTrain.csv")
x_test = pd.read_csv("C:/Users/User/Downloads/Credit Card Fraud  Dataset/fraudTest.csv")

CHECKING FOR BOTH MISSING AND DUPLICATE DATA

In [4]:
x_train.isnull().sum()
x_test.isnull().sum()
x_train.duplicated().sum()
x_test.duplicated().sum()

np.int64(0)

ENCODING CATEGORICAL FEATURES

In [5]:
x_train['job'] = x_train['job'].astype('category')
x_test['job'] = x_test['job'].astype('category')

x_train['job'] = x_train['job'].cat.codes
x_test['job'] = x_test['job'].cat.codes

In [6]:
le_gen = LabelEncoder()
x_train['gender'] = le_gen.fit_transform(x_train['gender'])
x_test['gender'] = le_gen.transform(x_test['gender'])



In [7]:
print(x_train.head())
print(x_test.head)

   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last  gender                        street  ...      lat      long  \
0    Banks       0                561 Perry Cove  ...  36.0788  -81.1781   
1     Gill       0  43039 Riley Greens Suite 393  ... 

In [8]:
print(x_train.shape)
print(x_test.shape)

print(x_train.head())
print(x_test.head())

(1296675, 23)
(555719, 23)
   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last  gender                        street  ...      lat      long  \
0    Banks       0                561 Perry Cove  ...  36.0788  -81.1781   
1     Gill       0  43039 R

FEATURE SCALING

In [9]:
scaler = StandardScaler()
numeric_columns = ['amt', 'lat', 'long', 'city_pop','unix_time']

existing_numeric_columns = [col for col in numeric_columns if col in x_train.columns]
x_train[existing_numeric_columns] = scaler.fit_transform(x_train[existing_numeric_columns])
x_test[existing_numeric_columns] = scaler.transform(x_test[existing_numeric_columns])

print(x_train[existing_numeric_columns].mean(), x_train[numeric_columns].std())
print(x_train.head())

amt         -9.447052e-18
lat         -3.796575e-16
long        -4.366253e-15
city_pop    -1.350753e-18
unix_time   -2.571352e-15
dtype: float64 amt          1.0
lat          1.0
long         1.0
city_pop     1.0
unix_time    1.0
dtype: float64
   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category       amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net -0.407826   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  0.230039  Stephanie   
2                fraud_Lind-Buckridge  entertainment  0.934149     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport -0.158132     Jeremy   
4                 fraud_Kee

In [10]:
columns_to_drop = ['Unnamed: 0', 'cc_num', 'street', 'city', 'state', 'zip']
x_train = x_train.drop(columns=[col for col in columns_to_drop if col in x_train.columns])
x_test = x_test.drop(columns=[col for col in columns_to_drop if col in x_test.columns])

print("Updated X_train shape:", x_train.shape)
print("Updated X_test shape:", x_test.shape)
print(x_train.head())


Updated X_train shape: (1296675, 17)
Updated X_test shape: (555719, 17)
  trans_date_trans_time                            merchant       category  \
0   2019-01-01 00:00:18          fraud_Rippin, Kub and Mann       misc_net   
1   2019-01-01 00:00:44     fraud_Heller, Gutmann and Zieme    grocery_pos   
2   2019-01-01 00:00:51                fraud_Lind-Buckridge  entertainment   
3   2019-01-01 00:01:16  fraud_Kutch, Hermiston and Farrell  gas_transport   
4   2019-01-01 00:03:06                 fraud_Keeling-Crist       misc_pos   

        amt      first     last  gender       lat      long  city_pop  job  \
0 -0.407826   Jennifer    Banks       0 -0.484420  0.657620 -0.282589  370   
1  0.230039  Stephanie     Gill       0  2.039120 -2.033870 -0.293670  428   
2  0.934149     Edward  Sanchez       1  0.717754 -1.601537 -0.280406  307   
3 -0.158132     Jeremy    White       1  1.515617 -1.590766 -0.287742  328   
4 -0.177094      Tyler   Garcia       1 -0.023035  0.782279 -0.293835

In [11]:
print(x_train.dtypes)

trans_date_trans_time     object
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                     int64
lat                      float64
long                     float64
city_pop                 float64
job                        int16
dob                       object
trans_num                 object
unix_time                float64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object


In [12]:
x_train = x_train.drop(columns=['dob', 'trans_num'])
x_test =  x_test.drop(columns=['dob', 'trans_num'])

print(x_train.dtypes)

trans_date_trans_time     object
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                     int64
lat                      float64
long                     float64
city_pop                 float64
job                        int16
unix_time                float64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object


In [13]:
categorical_columns = ['merchant', 'category', 'job']

for col in categorical_columns:
    le = LabelEncoder()
    x_train[col] = le.fit_transform(x_train[col])
    x_test[col] = le.transform(x_test[col])

columns_to_drop = ['first','last','trans_date_trans_time']
x_train = x_train.drop([col for col in columns_to_drop if col in x_train.columns], axis =1)
x_test = x_test.drop([col for col in columns_to_drop if col in x_test.columns], axis = 1)
print(x_train.head)

<bound method NDFrame.head of          merchant  category       amt  gender       lat      long  city_pop  \
0             514         8 -0.407826       0 -0.484420  0.657620 -0.282589   
1             241         4  0.230039       0  2.039120 -2.033870 -0.293670   
2             390         0  0.934149       1  0.717754 -1.601537 -0.280406   
3             360         2 -0.158132       1  1.515617 -1.590766 -0.287742   
4             297         9 -0.177094       1 -0.023035  0.782279 -0.293835   
...           ...       ...       ...     ...       ...       ...       ...   
1296670       499         0 -0.341769       1 -0.161575 -1.617214 -0.293309   
1296671         2         1 -0.116339       1  0.143638  0.924207 -0.293832   
1296672       599         1  0.221930       1 -1.102883 -1.133257 -0.291186   
1296673       509         1  0.028375       1  0.948613 -0.895029 -0.290434   
1296674       370         1 -0.412005       1  1.439314 -1.718754 -0.293441   

         job  unix_ti

In [14]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())

merchant      0
category      0
amt           0
gender        0
lat           0
long          0
city_pop      0
job           0
unix_time     0
merch_lat     0
merch_long    0
is_fraud      0
dtype: int64
merchant      0
category      0
amt           0
gender        0
lat           0
long          0
city_pop      0
job           0
unix_time     0
merch_lat     0
merch_long    0
is_fraud      0
dtype: int64


SPLITTING THE DATASET

In [15]:
y_train = x_train['is_fraud']
y_test = x_test['is_fraud']

x_train = x_train.drop(columns=['is_fraud'])
x_test = x_test.drop(columns=['is_fraud'])

print("X_train shape:", x_train.shape)
print("Y_train shape:", y_train.shape)
print("X_test shape:", x_test.shape)
print("Y_test shape:", y_test.shape)

X_train shape: (1296675, 11)
Y_train shape: (1296675,)
X_test shape: (555719, 11)
Y_test shape: (555719,)


RANDOM FOREST MDOEL

In [16]:
binary_columns = x_train.columns[x_train.nunique()== 2]
print(binary_columns)


Index(['gender'], dtype='object')


In [17]:
rf_model = RandomForestClassifier(n_estimators=100, random_state = 42, class_weight = 'balanced')


rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.4f}")

Accuracy: 99.6939


In [18]:
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test,y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.97      0.21      0.35      2145

    accuracy                           1.00    555719
   macro avg       0.98      0.61      0.67    555719
weighted avg       1.00      1.00      1.00    555719


Confusion Matrix:
 [[553559     15]
 [  1686    459]]
